In [5]:
from fetch import fetch
import ast

In [6]:
# convert source code into ast, a tree representation
with open('hello.py') as f: 
    source = f.read()

tree = ast.parse(source)
tree.body

In [3]:
# a flattened view of the ast - this is only for illustrative purposes and we wont use this
for statement in tree.body:
    print(ast.dump(statement), '\n')

Import(names=[alias(name='ast', asname=None)]) 

FunctionDef(name='a', args=arguments(args=[], vararg=None, kwonlyargs=[], kw_defaults=[], kwarg=None, defaults=[]), body=[Assign(targets=[Name(id='x', ctx=Store())], value=Num(n=2)), Return(value=BinOp(left=BinOp(left=Call(func=Name(id='c', ctx=Load()), args=[], keywords=[]), op=Add(), right=Call(func=Name(id='b', ctx=Load()), args=[], keywords=[])), op=Add(), right=Name(id='x', ctx=Load())))], decorator_list=[], returns=None) 

FunctionDef(name='b', args=arguments(args=[], vararg=None, kwonlyargs=[], kw_defaults=[], kwarg=None, defaults=[]), body=[Return(value=BinOp(left=Num(n=2), op=Add(), right=Call(func=Name(id='c', ctx=Load()), args=[], keywords=[])))], decorator_list=[], returns=None) 

FunctionDef(name='c', args=arguments(args=[], vararg=None, kwonlyargs=[], kw_defaults=[], kwarg=None, defaults=[]), body=[Return(value=Num(n=3))], decorator_list=[], returns=None) 

FunctionDef(name='d', args=arguments(args=[], vararg=None, kwonlyar

# [example](https://suhas.org/function-call-ast-python/) - carve out call graph
starting with the assign statement `y = a()` we will extract a call graph from source.

we first find the ast node corresponding to `y`.

In [4]:
# recursively find all callables, starting from y=a()
fetch("y", tree)

>> copying code for y ... 
y = a()

>> copying code for a ... 
def a():
    x = 2
    return c() + b() + x

>> copying code for b ... 
def b():
    return 2 + c()

>> copying code for c ... 
def c():
    return 3

